In [9]:
import hsfs  # A  import a new module to connect to hopsworks

connection = hsfs.connection(
    host="a8f80260-d911-11ec-84b0-47e953c88cbd.cloud.hopsworks.ai",  # B
    project="day_trading",  # C
    api_key_value="s1MMMjIFfOYBBMZ6.2fvt6qFHcAWNE8JDLeaWrfni6bGeR7sjiqUpfNLNtnQ5CJVN8x9iJA2df4v0Oz0L"  # D
)

fs = connection.get_feature_store(name='day_trading_featurestore')  # E

# A Connect to the feature store

# A
# B the host of your instance
# C the name of the project. Mine is 'day_trading'
# D the API key. this isn't mine..
# E Connect to the feature store


FeatureStoreException: Trying to instantiate Hive as engine, but 'hive' extras are missing in HSFS installation. Install with `pip install hsfs[hive]`.

In [6]:
import datetime
import pandas as pd

day_trading_features = pd.read_csv('../data/fifteen_percent_gains_features.csv')  # A
day_trading_features['symbol'] = 'TWLO'  # B

day_trading_features['date'] = day_trading_features['date'].apply(lambda x:datetime.datetime.strptime(x[:-6], '%Y-%m-%d %H:%M:%S')) # C

day_trading_features.set_index('date', inplace=True) # C

day_trading_features = day_trading_features.tz_localize('US/Pacific') # C

day_trading_features['date'] = day_trading_features.index.astype(int) # C

day_trading_features.head()


# A Read in the feature set from one of our best performing models from the day trading case study
# B Add a feature called symbol to signal that these refer to TWLO
# C Set our index to be our date feature and localize to US/Pacific


,close,volume,symbol,date
date,,,,
2020-01-02 14:30:00-08:00,99.98,93417.0,TWLO,1578004200000000000
2020-01-02 14:31:00-08:00,99.78,16685.0,TWLO,1578004260000000000
2020-01-02 14:32:00-08:00,100.14,21998.0,TWLO,1578004320000000000
2020-01-02 14:33:00-08:00,100.35,18348.0,TWLO,1578004380000000000
2020-01-02 14:34:00-08:00,100.55,22181.0,TWLO,1578004440000000000


In [7]:
day_trading_features[['symbol', 'date']].head()

,symbol,date
date,,
2020-01-02 14:30:00-08:00,TWLO,1578004200000000000
2020-01-02 14:31:00-08:00,TWLO,1578004260000000000
2020-01-02 14:32:00-08:00,TWLO,1578004320000000000
2020-01-02 14:33:00-08:00,TWLO,1578004380000000000
2020-01-02 14:34:00-08:00,TWLO,1578004440000000000


In [17]:
day_trading_features.head().to_dict()

{'close': {Timestamp('2020-01-02 14:30:00-0800', tz='US/Pacific'): 99.98,
  Timestamp('2020-01-02 14:31:00-0800', tz='US/Pacific'): 99.78,
  Timestamp('2020-01-02 14:32:00-0800', tz='US/Pacific'): 100.14,
  Timestamp('2020-01-02 14:33:00-0800', tz='US/Pacific'): 100.35,
  Timestamp('2020-01-02 14:34:00-0800', tz='US/Pacific'): 100.55},
 'volume': {Timestamp('2020-01-02 14:30:00-0800', tz='US/Pacific'): 93417.0,
  Timestamp('2020-01-02 14:31:00-0800', tz='US/Pacific'): 16685.0,
  Timestamp('2020-01-02 14:32:00-0800', tz='US/Pacific'): 21998.0,
  Timestamp('2020-01-02 14:33:00-0800', tz='US/Pacific'): 18348.0,
  Timestamp('2020-01-02 14:34:00-0800', tz='US/Pacific'): 22181.0},
 'symbol': {Timestamp('2020-01-02 14:30:00-0800', tz='US/Pacific'): 'TWLO',
  Timestamp('2020-01-02 14:31:00-0800', tz='US/Pacific'): 'TWLO',
  Timestamp('2020-01-02 14:32:00-0800', tz='US/Pacific'): 'TWLO',
  Timestamp('2020-01-02 14:33:00-0800', tz='US/Pacific'): 'TWLO',
  Timestamp('2020-01-02 14:34:00-0800', tz

In [11]:
try:
    price_volume_features = fs.get_feature_group('price_volume_features')
    price_volume_features.delete()
except:
    pass

try:
    twitter_fg = fs.get_feature_group('twitter_features')
    twitter_fg.delete()
except:
    pass

In [12]:
twitter_fg = fs.create_feature_group(name="twitter_features",  # A
                                    primary_key=["symbol"],  # B
                                    event_time = "date",  # C
                                    version=1,  # D
                                    description="Twitter Features", # E
                                    online_enabled=True) # F

twitter_features = day_trading_features[['symbol', 'date', 'feature__rolling_7_day_total_tweets', 'feature__rolling_1_day_verified_count']]  # G
twitter_fg.save(twitter_features) # G

# A the name of our feature group
# B the primary key of our table identifies the entities in the data
# C the event_time tells the feature store which data points are more recent
# D We can version features but this won't be covered in this text
# E A brief description of the feature group
# F Enabling online feature groups gives us the ability to get the most recent feature values quickly
# G select the twitter features and save them to our feature group


Configuring ingestion job...
Uploading Pandas dataframe...
Launching ingestion job...
Ingestion Job started successfully, you can follow the progress at https://40183d50-5e14-11ec-8b98-916189484d30.cloud.hopsworks.ai/hopsworks/#!/project/119/jobs


In [3]:
twitter_fg = fs.get_feature_group(name="twitter_features", version='1')  # A

data_from_fs_twitter = twitter_fg.read()  # A

data_from_fs_twitter.head()

# A Get our data out of the feature store as a pandas dataframe

,fg0.symbol,fg0.date,fg0.feature__rolling_7_day_total_tweets,fg0.feature__rolling_1_day_verified_count
0,TWLO,1605798000000000000,481.0,0.0
1,TWLO,1611676800000000000,365.0,1.0
2,TWLO,1585663200000000000,292.0,0.0
3,TWLO,1619791200000000000,397.0,0.0
4,TWLO,1593712800000000000,749.0,0.0


In [4]:
twitter_fg.show(2)  # A

# A Another way to view rows

,fg0.symbol,fg0.date,fg0.feature__rolling_7_day_total_tweets,fg0.feature__rolling_1_day_verified_count
0,TWLO,1605798000000000000,481.0,0.0
1,TWLO,1611676800000000000,365.0,1.0


In [15]:
price_volume_fg = fs.create_feature_group(name="price_volume_features",  # A
                                    primary_key=["symbol"],  # B
                                    event_time = "date",  # C
                                    version=1,  # D
                                    description="Price and Volume Features", # E
                                    online_enabled=True) # F

price_volume_fg.save(day_trading_features.drop(['feature__rolling_7_day_total_tweets', 'feature__rolling_1_day_verified_count'], axis=1))  # G
twitter_fg.save(twitter_features) # G

# A the name of our feature group
# B the primary key of our table identifies the entities in the data
# C the event_time tells the feature store which data points are more recent
# D We can version features but this won't be covered in this text
# E A brief description of the feature group
# F Enabling online feature groups gives us the ability to get the most recent feature values quickly
# G select the non-twitter features and save them to our feature group


Configuring ingestion job...
Uploading Pandas dataframe...
Launching ingestion job...
Ingestion Job started successfully, you can follow the progress at https://40183d50-5e14-11ec-8b98-916189484d30.cloud.hopsworks.ai/hopsworks/#!/project/119/jobs


RestAPIError: Metadata operation error: (url: https://40183d50-5e14-11ec-8b98-916189484d30.cloud.hopsworks.ai/hopsworks-api/api/project/119/featurestores/67/featuregroups). Server response: 
HTTP code: 400, HTTP reason: Bad Request, error code: 270089, error msg: The feature group you are trying to create does already exist., user msg: project: day_trading, featurestoreId: 67

In [16]:
price_volume_fg = fs.get_feature_group(name="price_volume_features")

# Get our data out of the feature store
data_from_fs_price = price_volume_fg.read()

print(type(data_from_fs_price))

data_from_fs_price.head()


<class 'pandas.core.frame.DataFrame'>


,fg0.feature__dayofweek,fg0.feature__morning,fg0.feature__lag_30_min_ago_price,fg0.feature__lag_7_day_ago_price,fg0.feature__rolling_close_mean_60,fg0.feature__rolling_close_std_60,fg0.feature__rolling_volume_mean_60,fg0.feature__rolling_volume_std_60,fg0.feature__expanding_close_mean,fg0.feature__expanding_volume_mean,fg0.feature__overnight_change_close,fg0.feature__monthly_pct_change_close,fg0.feature__expanding_average_close,fg0.feature__macd,fg0.symbol,fg0.date
0,3,True,196.335,196.68,195.828505,0.459430,3635.366667,2982.815210,120.503476,9358.597413,0.010239,1.062370,120.155650,22.394148,TWLO,1590688800000000000
1,4,True,243.790,223.09,243.922153,0.369915,2160.850000,1400.073095,173.926178,7890.282344,-0.008211,-0.036595,173.613474,-4.583344,TWLO,1601056800000000000
2,0,True,349.100,362.63,351.314473,2.255349,5746.033333,4179.435693,207.647944,7427.689615,-0.015320,0.338683,209.085272,16.335508,TWLO,1609171200000000000
3,0,True,366.630,366.39,359.210687,4.061210,9210.935484,11013.576476,207.544914,7428.895607,-0.015320,0.338683,209.085272,16.335508,TWLO,1609167600000000000
4,2,False,190.992,184.77,191.304752,0.412972,9218.200000,5665.613234,116.322433,9182.972136,0.040958,1.315708,115.903912,20.469852,TWLO,1590001200000000000


In [5]:
price_volume_fg = fs.get_feature_group(name="price_volume_features")

price_volume_fg.read(online=True)  # A

# A only get the most recent datapoint given our 

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '10.0.0.219' (timed out)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# TODO get online of each fg

In [6]:
twitter_fg.select(['feature__rolling_7_day_total_tweets']).show(2)

,fg0.feature__rolling_7_day_total_tweets
0,481.0
1,365.0


In [7]:
query = twitter_fg.select_all().join(price_volume_fg.select_all(), on=['date', 'symbol'])  # A

query.show(2)  # B

# A combine our two feature groups to reconstruct our original data
# B show our joined data

,right_fg0.symbol,right_fg0.date,right_fg0.feature__rolling_7_day_total_tweets,right_fg0.feature__rolling_1_day_verified_count,right_fg0.feature__dayofweek,right_fg0.feature__morning,right_fg0.feature__lag_30_min_ago_price,right_fg0.feature__lag_7_day_ago_price,right_fg0.feature__rolling_close_mean_60,right_fg0.feature__rolling_close_std_60,right_fg0.feature__rolling_volume_mean_60,right_fg0.feature__rolling_volume_std_60,right_fg0.feature__expanding_close_mean,right_fg0.feature__expanding_volume_mean,right_fg0.feature__overnight_change_close,right_fg0.feature__monthly_pct_change_close,right_fg0.feature__expanding_average_close,right_fg0.feature__macd
0,TWLO,1582815600000000000,213.0,0.0,3,True,107.52,126.93,108.297719,0.912541,26119.032258,40410.254492,122.554386,7602.227133,-0.032747,-0.023526,122.206249,1.504137
1,TWLO,1582819200000000000,216.0,1.0,3,True,108.19,128.20,109.732097,0.759198,9841.350000,6172.054963,122.485363,7614.280549,-0.032747,-0.023526,122.206249,1.504137


In [ ]:
td = fs.get_training_dataset('training_data', version=1)  # A

training_df = td.read(split='training')  # B
print(training_df.shape)  # B

testing_df = td.read(split='testing')  # C
print(testing_df.shape)  # C

# A Select our training dataset from Hopsworks
# B read and print the shape of our training set
# C read and print the shape of our testing set

2022-01-03 08:50:44,935 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-01-03 08:51:05,406 INFO ipc.Client: Retrying connect to server: 10.0.0.219/10.0.0.219:8020. Already tried 0 time(s); maxRetries=45
2022-01-03 08:51:25,409 INFO ipc.Client: Retrying connect to server: 10.0.0.219/10.0.0.219:8020. Already tried 1 time(s); maxRetries=45
2022-01-03 08:51:45,414 INFO ipc.Client: Retrying connect to server: 10.0.0.219/10.0.0.219:8020. Already tried 2 time(s); maxRetries=45
2022-01-03 08:52:05,418 INFO ipc.Client: Retrying connect to server: 10.0.0.219/10.0.0.219:8020. Already tried 3 time(s); maxRetries=45
2022-01-03 08:52:25,421 INFO ipc.Client: Retrying connect to server: 10.0.0.219/10.0.0.219:8020. Already tried 4 time(s); maxRetries=45
2022-01-03 08:52:45,424 INFO ipc.Client: Retrying connect to server: 10.0.0.219/10.0.0.219:8020. Already tried 5 time(s); maxRetries=45
2022-01-03 08:53:05,427 INFO i

In [ ]:
price_volume_fg = fs.get_feature_group(name="price_volume_features")

price_volume_fg.read(online=True)  # A

# A only get the most recent datapoint given our 

In [ ]:
connection.close()